In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
import plotly.express as px
import time
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import  metrics


In [3]:
## Utility Function

# Refer :- https://www.kaggle.com/rinnqd/reduce-memory-usage

def reduce_memory_usage(df):
  
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Reading merged application file

In [4]:
app_merged_df = reduce_memory_usage(pd.read_csv('application_merged.csv'))
print(app_merged_df.shape)
app_merged_df.head()

Memory usage of dataframe is 1283.33 MB
Memory usage after optimization is: 373.03 MB
Decreased by 70.9%
(307511, 547)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,CR_PAYMENT_DIV_MIN_VAR,CR_LATE_PAYMENT_MAX,CR_LATE_PAYMENT_MEAN,CR_LATE_PAYMENT_SUM,CR_LATE_PAYMENT_VAR,CR_DRAWING_LIMIT_RATIO_MAX,CR_DRAWING_LIMIT_RATIO_MEAN,CR_DRAWING_LIMIT_RATIO_SUM,CR_DRAWING_LIMIT_RATIO_VAR,CREDIT_COUNT
0,100002,1,0,"202,500.00","406,597.50","24,700.50","351,000.00",0.02,-9461,-637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,0,"270,000.00","1,293,502.50","35,698.50","1,129,500.00",0.00,-16765,-1188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,"67,500.00","135,000.00","6,750.00","135,000.00",0.01,-19046,-225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,"135,000.00","312,682.50","29,686.50","297,000.00",0.01,-19005,-3039,...,NaN,0.00,0.00,0.00,0.00,NaN,NaN,0.00,NaN,6.00
4,100007,0,0,"121,500.00","513,000.00","21,865.50","513,000.00",0.03,-19932,-3038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
features = app_merged_df.drop(['TARGET'], axis=1)

label = app_merged_df['TARGET']

print(features.shape)
print(label.shape)

(307511, 546)
(307511,)


### Train/Test Split

In [6]:
X_train,X_test, y_train, y_test = train_test_split(features, label,  test_size=0.2, random_state=123)

print ("X_train :" , X_train.shape )
print ("X_test :", X_test.shape)
print ("y_train :", y_train.shape)
print ("y_test :", y_test.shape)

X_train : (246008, 546)
X_test : (61503, 546)
y_train : (246008,)
y_test : (61503,)


In [7]:
X_train_final = np.nan_to_num(X_train)
X_test_final = np.nan_to_num(X_test)

print ("X_train_final :" , X_train_final.shape )
print ("X_test_final :", X_test_final.shape)

X_train_final : (246008, 546)
X_test_final : (61503, 546)


### Feature Selection

#### KBest

In [8]:
from sklearn.feature_selection import SelectKBest

number_of_features_to_select = 400

feature_selector = SelectKBest(k=number_of_features_to_select)

feature_selector = feature_selector.fit(X_train_final, y_train)

cols = feature_selector.get_support(indices = True)

X_train_best = X_train.iloc[:,cols]
X_test_best = X_test.iloc[:,cols]

print ("X_train_best :" , X_train_best.shape )
print ("X_test_best :", X_test_best.shape)

/opt/anaconda3/envs/pyds/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/anaconda3/envs/pyds/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/anaconda3/envs/pyds/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/anaconda3/envs/pyds/lib/python3.10/site-packages/sklearn/utils/__init__.py:711: RuntimeWarning: overflow encountered in square
  X = X**2
/opt/anaconda3/envs/pyds/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/anaconda3/envs/pyds/lib/python3.10/site-packages/sklearn/utils/__init__.

X_train_best : (246008, 400)
X_test_best : (61503, 400)


### Standarding the final dataset

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train_best)
X_test_std = scaler.fit_transform(X_test_best)

#replacing nan values with 0
X_train_std[np.isnan(X_train_std)] = 0
X_test_std[np.isnan(X_test_std)] = 0

print ("X_train_std :" , X_train_std.shape )
print ("X_test_std :", X_test_std.shape)

X_train_std : (246008, 400)
X_test_std : (61503, 400)


## Light GBM

### Hyperparameter Tuning using Bayesian Optimization

In [10]:
!pip install bayesian-optimization

In [11]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [12]:
lgb = LGBMClassifier()

In [13]:
results = []
names = []
## for classification tasks
scoring_metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

In [ ]:
model = lgb.fit (X_train_std, np.ravel(y_train))

In [ ]:
## STOP here

prin(fail)

### Model Evaluation Summary

In [8]:
algo = ["LR", "RF", "SVC", "GB", "KNN"]
d1 = {'algo': algo,
     'roc_auc_score': roc_auc_score,
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score,
    }
df1 = pd.DataFrame(data=d1)
df1

,algo,roc_auc_score,f1_score,accuracy_score
0,LR,0.51,0.05,0.92
1,RF,0.50,0.00,0.92
2,SVC,0.50,0.01,0.92
3,GB,0.51,0.06,0.92
4,KNN,0.51,0.04,0.91


In [9]:
import pickle
model_file = 'model_all.pkl'
with open(model_file,'wb') as f:
    pickle.dump(model,f)
print ("model saved to : ", model_file)

model saved to :  model_all.pkl


### Feature Selection

In [ ]:
import lightgbm as lgb

model_sk = lgb.LGBMClassifier(boosting_type='gbdt', max_depth=7, learning_rate=0.01, n_estimators= 2000, 
                 class_weight='balanced', subsample=0.9, colsample_bytree= 0.8, n_jobs=-1)
train_features, valid_features, train_y, valid_y = train_test_split(X_train, y_train, test_size = 0.15, random_state = 42)
model_sk.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], eval_metric = 'auc')

/opt/anaconda3/envs/pyds/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


In [ ]:
feature_imp = pd.DataFrame(sorted(zip(model_sk.feature_importances_, X_train_final.columns)), columns=['Value','Feature'])
features_df = feature_imp.sort_values(by="Value", ascending=False)
selected_features = list(features_df[features_df['Value']>=50]['Feature'])
# Saving the selected features into pickle file
with open('select_features.txt','wb') as fp:
    pickle.dump(selected_features, fp)
print('The no. of features selected:',len(selected_features))

In [ ]:
# Feature importance Plot
data1 = features_df.head(20)
data = [go.Bar(x =data1.sort_values(by='Value')['Value'] , y = data1.sort_values(by='Value')['Feature'], orientation = 'h',
              marker = dict(
        color = 'rgba(43, 13, 150, 0.6)',
        line = dict(
            color = 'rgba(43, 13, 150, 1.0)',
            width = 1.5)
    )) ]
layout = go.Layout(
    autosize=False,
    width=1300,
    height=700,
    title = "Top 20 important features",
    xaxis=dict(
        title='Importance value'
        ),
    yaxis=dict(
        automargin=True
        ),
    bargap=0.4
    )
fig = go.Figure(data = data, layout=layout)
fig.layout.template = 'seaborn'
py.iplot(fig)